In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pyomo.environ import *
import pyomo.environ as pyo
from idaes.core.util.model_statistics import degrees_of_freedom
from pyomo.util.calc_var_value import calculate_variable_from_constraint
from Method import Initialize
from Unit_Model import (H101_creater,R101_creater,F101_creater)
from pyomo.network import *

In [2]:
# Create a concrete model
m = ConcreteModel()

# Define a set for components
m.Phase = Set(initialize = ['Liq','Vap'])
m.Comp = Set(initialize = ['benzene','toluene','methane','hydrogen'])
m.R101 = R101_creater(m)
m.H101 = H101_creater(m)

In [3]:
# 各组分进料
m.H101.inlet.flow['Liq','benzene'].fix(2.0008e-05)
m.H101.inlet.flow['Liq','toluene'].fix(0.30001)
m.H101.inlet.flow['Liq','methane'].fix(2.0008e-05)
m.H101.inlet.flow['Liq','hydrogen'].fix(2.0008e-05)
m.H101.inlet.flow['Vap','benzene'].fix(0.34136)
m.H101.inlet.flow['Vap','toluene'].fix(0.033075)
m.H101.inlet.flow['Vap','methane'].fix(1.0994)
m.H101.inlet.flow['Vap','hydrogen'].fix(0.50088)

# 温度压力
m.H101.inlet.P.fix(350000)
m.H101.inlet.T.fix(330.80)

m.H101.outlet.P.fix(350000)
m.H101.outlet.T.fix(600)

In [4]:
m.s1 = Arc(source=m.H101.Outlet, destination=m.R101.Inlet)

In [5]:
#m.F101.outlet.P.fix(350000)
#m.F101.outlet.T.fix(350)

m.R101.Q.fix(0)

In [8]:
TransformationFactory("network.expand_arcs").apply_to(m)

In [9]:
degrees_of_freedom(m)

0

In [10]:
Initialize(m.H101)

In [11]:
# 模型求解
solver = pyo.SolverFactory('ipopt')
solver.options['max_iter'] = 6000
results = solver.solve(m,tee=True)

Ipopt 3.13.2: max_iter=6000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computatio

In [12]:
m.H101.pprint()

H101 : Size=1, Index=None, Active=True
    7 Set Declarations
        Comp : Size=1, Index=None, Ordered=Insertion
            Key  : Dimen : Domain : Size : Members
            None :     1 :    Any :    4 : {'benzene', 'toluene', 'methane', 'hydrogen'}
        Phase : Size=1, Index=None, Ordered=Insertion
            Key  : Dimen : Domain : Size : Members
            None :     1 :    Any :    2 : {'Liq', 'Vap'}
        cp_ig_1_index : Size=1, Index=None, Ordered=True
            Key  : Dimen : Domain               : Size : Members
            None :     2 : H101.Phase*H101.Comp :    8 : {('Liq', 'benzene'), ('Liq', 'toluene'), ('Liq', 'methane'), ('Liq', 'hydrogen'), ('Vap', 'benzene'), ('Vap', 'toluene'), ('Vap', 'methane'), ('Vap', 'hydrogen')}
        cp_ig_2_index : Size=1, Index=None, Ordered=True
            Key  : Dimen : Domain               : Size : Members
            None :     2 : H101.Phase*H101.Comp :    8 : {('Liq', 'benzene'), ('Liq', 'toluene'), ('Liq', 'methane'), 